In [ ]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 72.7 MB/s eta 0:00:00


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
print(torch.cuda.is_available())


NameError: name 'torch' is not defined

In [ ]:
import torch

# Create two matrices
A = torch.rand(2,1, 4)
B = torch.rand(2,4, 1)

# Multiply the matrices
C = torch.matmul(A, B)
print(A)
print(B)
print(C)

# Print the result
print(C.shape)

ModuleNotFoundError: No module named 'torch'

# Imports

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# Encoder

In [ ]:
model = models.vgg19(pretrained=True)

print(model.features)

layer = model.features[10]

weights = layer.weight.detach().cpu().numpy()

print(weights.shape)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

# KNN

In [ ]:
def knn_graph(x, y, k):
    '''
    for each node in y, search for k-nearest-neighbor in x
    :param x: BxNxE tensor, B batches, N nodes, E embeddings
    :param y: BxMxE tensor, B batches, M nodes, E embeddings
    :param k: k nearest neighbors
    :return idx_k: BxMxK tensor, B batches, M node indices in y, K neighboring node indices in x
    '''

    D = normalized_cross_correlation(y, x.permute(0,2,1))

    # print(D)
    # print(D.shape)

    _, idx_k = torch.topk(D, k, dim=2, largest=True, sorted=True)
    # print(idx_k)
    # print(idx_k.shape)
    return idx_k

def normalized_cross_correlation(x, y, eps=1e-8):
    dev_xy = torch.matmul(x,y)
    dev_xx = torch.mul(x,x)
    dev_yy = torch.mul(y,y)

    dev_xx_sum = torch.sum(dev_xx, dim=-1, keepdim=True)
    dev_yy_sum = torch.sum(dev_yy, dim=-2, keepdim=True)

    ncc = torch.div(dev_xy + eps, torch.sqrt( torch.mul(dev_xx_sum, dev_yy_sum)) + eps)
    return ncc

knn_graph(torch.rand(2,7,10),torch.rand(2,4, 10),3)


tensor([[[6, 3, 2],
         [6, 3, 2],
         [2, 3, 6],
         [6, 2, 3]],

        [[5, 1, 0],
         [2, 0, 1],
         [5, 1, 2],
         [5, 4, 1]]])

# GAT layer

In [ ]:
import dgl
import dgl.nn.pytorch as dglnn
import torch.nn.functional as F

class HeterConv(nn.Module):
    def __init__(self, feat_c, patch_size):
        super().__init__()
        self.feat_c = feat_c
        self.patch_size = patch_size
        self.feat_pc = feat_c*patch_size*patch_size

        # GATv2
        self.conv1 = dglnn.GATv2Conv(self.feat_pc, self.feat_pc, num_heads=1, allow_zero_in_degree=True)
        self.conv2 = dglnn.GATv2Conv(self.feat_pc, self.feat_pc, num_heads=1, allow_zero_in_degree=True)

        # SwinIR
        # self.conv1 = dglnn.TWIRLSConv(self.feat_pc, self.feat_pc, hidden_d = 64, prop_step = 2)
        # self.conv2 = dglnn.TWIRLSConv(self.feat_pc, self.feat_pc, hidden_d = 64, prop_step = 2)

    def forward(self, feat_c, feat_s, idx_k1, idx_k2):
        b,n,f = feat_s.shape
        # print(feat_s.shape) torch.Size([8, 784, 12800]) 12800=512*5*5 784=28*28
        b,m,f = feat_c.shape
        _,_,k = idx_k1.shape
        c = self.feat_c; p = self.patch_size
        dev=feat_c.device

        torch.cuda.empty_cache()

        # indices
        idx_v = torch.tensor(range(m), device=dev, dtype=torch.int64).view(1,m,1).expand(2,m,k).contiguous().view(-1)
        print(idx_v)
        # feat of content patch og
        idx_u = torch.cat((idx_k2, idx_k1+m), dim=1).contiguous().view(b,-1)
        print(idx_u)


        # features
        feat = torch.cat((feat_c, feat_s), dim=1)

        agg = torch.tensor([]).to(dev)
        for i in range(b):
            # create graph
            graph = dgl.graph((idx_u[i], idx_v), num_nodes=m+n)
            if i == 0:
                print(graph.device)
            feat_out = self.conv(graph, feat[i])[:m] # m,f
            agg = torch.cat((agg, feat_out.contiguous().view(1,m,f)), dim=0)

        return agg

    def conv(self, graph, x):
        h = self.conv1(graph, x)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

h = HeterConv(1,3)
print(next(h.conv1.parameters()).device)
# batch, nb patch, nb feat
feat_c = torch.rand(2,8,9)
feat_s = torch.rand(2,8,9)
idx_k1 = knn_graph(feat_s, feat_c, 3)
idx_k2 = knn_graph(feat_c, feat_c, 3)
print("s to c")
print(idx_k1)
print("c to c")
print(idx_k2)
print("--------------------")
out = h(feat_c, feat_s, idx_k1, idx_k2)
print(out)
print(out.shape)


cpu
s to c
tensor([[[1, 2, 5],
         [2, 4, 7],
         [1, 3, 7],
         [1, 5, 3],
         [4, 2, 5],
         [4, 7, 1],
         [4, 1, 5],
         [5, 4, 2]],

        [[2, 0, 4],
         [7, 6, 4],
         [4, 2, 0],
         [1, 4, 6],
         [4, 3, 0],
         [3, 7, 2],
         [1, 4, 5],
         [3, 7, 0]]])
c to c
tensor([[[0, 3, 1],
         [1, 4, 0],
         [2, 0, 5],
         [3, 0, 4],
         [4, 7, 1],
         [5, 2, 3],
         [6, 4, 7],
         [7, 4, 6]],

        [[0, 2, 4],
         [1, 5, 3],
         [2, 0, 5],
         [3, 1, 7],
         [4, 0, 6],
         [5, 1, 2],
         [6, 3, 2],
         [7, 3, 5]]])
--------------------
tensor([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7,
        0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7])
tensor([[ 0,  3,  1,  1,  4,  0,  2,  0,  5,  3,  0,  4,  4,  7,  1,  5,  2,  3,
          6,  4,  7,  7,  4,  6,  9, 10, 13, 10, 12, 15,  9, 11, 15,  9

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


# ADAIN

In [ ]:
import torch

def calc_mean_std(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std


def adaptive_instance_normalization(content_feat, style_feat):
    assert (content_feat.size()[:2] == style_feat.size()[:2])
    size = content_feat.size()
    style_mean, style_std = calc_mean_std(style_feat)
    content_mean, content_std = calc_mean_std(content_feat)

    normalized_feat = (content_feat - content_mean.expand(
        size)) / content_std.expand(size)
    return normalized_feat * style_std.expand(size) + style_mean.expand(size)

# extra methodes for patch2feat and feat2 patch

In [ ]:
def calc_padding(x_shape, patchsize, stride, padding=None):
    if padding is None:
        xdim = x_shape
        padvert = -(xdim[0] - patchsize) % stride
        padhorz = -(xdim[1] - patchsize) % stride
        padtop = int(np.floor(padvert / 2.0))
        padbottom = int(np.ceil(padvert / 2.0))
        padleft = int(np.floor(padhorz / 2.0))
        padright = int(np.ceil(padhorz / 2.0))
    else:
        padtop = padbottom = padleft = padright = padding
    return padtop, padbottom, padleft, padright


# patch2feat (non functional)

In [ ]:
import torch

def extract_patches(images, patch_size, stride = 1):
    patches = []
    _, c, h, w = images.shape
    print(((h - patch_size + 1)//stride)**2)

    for image in images:
        tmp = []

        for y in range(0, h - patch_size + 1, stride):
            for x in range(0, w - patch_size + 1, stride):
                patch = image[:, y:y+patch_size, x:x+patch_size].reshape(-1)
                tmp.append(patch)
        patches.append(torch.stack(tmp))

    print(patches[0].shape)
    return torch.stack(patches)

image_batch = torch.randn(2, 3, 128, 128)
patch_size = 9
stride = 4

extracted_patches = extract_patches(image_batch, patch_size, stride)

print(extracted_patches.shape)

900
torch.Size([900, 243])
torch.Size([2, 900, 243])


# Functional Patch2feat with torch unfold

In [ ]:
import torch
import torch.nn as nn
def extract_patches(images, patch_size = 7, stride = 2):
    unfold = nn.Unfold(kernel_size=(patch_size, patch_size), stride = stride)
    res = unfold(images)
    b, e, n = res.shape
    return res.view(b,n,e)

image_batch = torch.randn(2, 3, 128, 128)
patch_size = 5

stride = 2

extracted_patches = extract_patches(image_batch, patch_size, stride)
print(extracted_patches.shape)

torch.Size([2, 3844, 75])


# Pyramid feature

In [ ]:
import torchvision.models as models
import torch
def pyramid_feature(image): # we use a 4 by 4 grid
  model = models.vgg19(pretrained=True)
  enc_1 = model.features[:18]  # relu3_1
  enc_2 = model.features[18:27]  #relu4_1
  feat_1 = enc_1(image)
  feat_2 = enc_2(feat_1)
  feat_1 = extract_patches(feat_1, 14, 2)
  feat_2 = extract_patches(feat_2, 7, 1)
  return torch.cat([feat_1,feat_2], dim = 2)

print(pyramid_feature(torch.randn(2, 3, 224, 224)).shape)

torch.Size([2, 484, 75264])


# feat2patch (non functnional)

In [ ]:
import torch

def reconstruct_image(patches, image_size, patch_size, patch_stride):
    reconstructed = torch.zeros((1, 1, image_size, image_size))  # Initialize the reconstructed image tensor

    patch_count = 0
    for y in range(0, image_size - patch_size + 1, patch_stride):
        for x in range(0, image_size - patch_size + 1, patch_stride):
            if patch_count < len(patches):  # Check if the patch count is within the valid range
                patch = patches[patch_count]
                reconstructed[:, :, y:y+patch_size, x:x+patch_size] += patch
                patch_count += 1

    return reconstructed / (patch_count // (patch_stride ** 2))  # Normalize the reconstructed image

# Example usage
image = torch.tensor([[1, 2, 3, 4, 5],
                      [6, 7, 8, 9, 10],
                      [11, 12, 13, 14, 15],
                      [16, 17, 18, 19, 20],
                      [21, 22, 23, 24, 25]], dtype=torch.float32)

patch_size = 3
patch_stride = 2

patches = extract_patches(image, patch_size, patch_stride)

image_size = len(image)
reconstructed_image = reconstruct_image(patches, image_size, patch_size, patch_stride)

print(reconstructed_image.squeeze(0).squeeze(0))  # Print the reconstructed image


# functional feat2patch with torch fold

In [ ]:
fold = nn.Fold(output_size=(4, 5), kernel_size=(2, 2))
input = torch.randn(3 * 2 * 2, 12)
output = fold(input)
output.size()

torch.Size([3, 4, 5])

In [ ]:
def reconstruct_image(patches, image_size, patch_size = 7, patch_stride = 2):
  b, n, e = patches.shape
  res = patches.view(b, e, n)
  counts = torch.ones_like(torch.empty(e, n))
  fold = nn.Fold(output_size=(image_size, image_size), kernel_size=(patch_size, patch_size), stride = patch_stride)
  counts = fold(counts)
  res = fold(res)
  return torch.div(res, counts)

image_batch = torch.randn(2, 3, 128, 128)
img_size = 128
patch_size = 5

stride = 2

extracted_patches = extract_patches(image_batch, patch_size, stride)
print(extracted_patches.shape)
rec_img = reconstruct_image(extracted_patches, img_size, patch_size, stride)
print(rec_img.shape)

torch.Size([2, 3844, 75])
torch.Size([2, 3, 128, 128])


# Construct the network

In [ ]:
decoder_block1 = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 256, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='bilinear'),
)
decoder_block2 = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 128, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='bilinear')
)
decoder_block3 = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 64, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='bilinear'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 3, (3, 3)),
)



class MyNet(nn.Module):
    def __init__(self, k, patch_size, stride, conv_type):
        super(MyNet, self).__init__()
        model = models.vgg19(pretrained=True)

        self.enc_1 = model[:4]  # input -> relu1_1
        self.enc_2 = model[4:11]  # relu1_1 -> relu2_1
        self.enc_3 = model[11:18]  # relu2_1 -> relu3_1
        self.enc_4 = model[18:27]  # relu3_1 -> relu4_1
        self.dec_1 = decoder_block1
        self.dec_2 = decoder_block2
        self.dec_3 = decoder_block3
        self.graph = GraphBlock(n_c=512, conv=conv_type, k=k, patch_size=patch_size, stride=stride)
        self.mse_loss = nn.MSELoss()

        self.update_layers = nn.Sequential(self.graph, self.dec_2, self.dec_3)

        # fix the encoder
        for name in ['enc_1', 'enc_2', 'enc_3', 'enc_4']:
            for param in getattr(self, name).parameters():
                param.requires_grad = False

    # extract relu1_1, relu2_1, relu3_1, relu4_1  from input image
    def encode_with_intermediate(self, input):
        results = [input]
        for i in range(4):
            func = getattr(self, 'enc_{:d}'.format(i + 1))
            results.append(func(results[-1]))
        return results[1:]

    # extract relu4_1 from input image
    def encode(self, input):
        for i in range(4):
            input = getattr(self, 'enc_{:d}'.format(i + 1))(input)
        return input

    def calc_content_loss(self, input, target):
        #assert (input.size() == target.size())
        return self.mse_loss(input, target)

    def calc_style_loss(self, input, target):
        #assert (input.size() == target.size())
        #assert (target.requires_grad is False)
        input_mean, input_std = calc_mean_std(input)
        target_mean, target_std = calc_mean_std(target)
        return self.mse_loss(input_mean, target_mean) + \
               self.mse_loss(input_std, target_std)

    def forward(self, content, style, alpha=1.0):
        # encoder
        assert 0 <= alpha <= 1
        style_feats = self.encode_with_intermediate(style)
        content_feat = self.encode(content)
        style_feat = self.encode(style)
        # decoder
        t = content_feat
        content_t = self.graph(style_feat, content_feat)
        content_t = adaptive_instance_normalization(content_t, style_feat)

        content_t = self.dec_2(content_t)
        output = self.dec_3(content_t)
        g_t_feats = self.encode_with_intermediate(output)

        # loss
        loss_c = self.calc_content_loss(g_t_feats[-1], t)
        loss_s = 0.
        for i in range(3):
            loss_s += self.calc_style_loss(g_t_feats[i], style_feats[i])
        return output, loss_c, loss_s